In [1]:
import pandas as pd
import numpy as np
import json

In [4]:
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.layouts import column, row, widgetbox
from bokeh.models import (CDSView, ColorBar, ColumnDataSource, CustomJS, CustomJSFilter,GeoJSONDataSource, HoverTool,LinearColorMapper, Slider)
from bokeh.palettes import brewer
from shapely.geometry import Point, Polygon

In [5]:
data = gpd.read_file('/home/eugene/Документы/jsonheat/spb1.geojson')
data = data[["NAME", "geometry"]]
data

,NAME,geometry
0,Металлострой,"POLYGON ((30.57063 59.81767, 30.57122 59.81742..."
1,округ Юго-Запад,"POLYGON ((30.23920 59.86393, 30.22867 59.85674..."
2,округ Московская застава,"POLYGON ((30.33814 59.90507, 30.33813 59.90500..."
3,округ Урицк,"POLYGON ((30.19716 59.84589, 30.20374 59.84576..."
4,округ Константиновское,"POLYGON ((30.12778 59.80916, 30.12769 59.80907..."
...,...,...
78,Народный округ,"POLYGON ((30.46394 59.88960, 30.46478 59.88977..."
79,Ивановский округ,"POLYGON ((30.43989 59.89175, 30.44436 59.89244..."
80,округ Невская застава,"POLYGON ((30.43989 59.89175, 30.43972 59.89172..."
81,округ № 54,"POLYGON ((30.47538 59.89231, 30.47410 59.89199..."


In [6]:
courses = pd.read_json("/home/eugene/Документы/jsonheat/courses_longlat.json", orient = 'collumns')
schools = pd.read_json("/home/eugene/Документы/jsonheat/edu_lonlat.json", orient = 'collumns')
schools.head()

,id,address,name,url,long,lat
0,1022453690,"Россия, Санкт-Петербург, 13-я линия Васильевск...",ГБОУ СОШ № 5,http://sch5.spb.ru/,30.272534,59.939229
1,1266587404,"Россия, Санкт-Петербург, Лермонтовский проспек...",Государственное бюджетное образовательное учре...,http://www.school280.ru/,30.298014,59.911844
2,1071875070,"Россия, Санкт-Петербург, набережная реки Фонта...",ГБОУ СОШ № 206,https://206school.ru/,30.339133,59.928704
3,1030553340,"Россия, Санкт-Петербург, Опочинина улица, 10",ГБОУ СОШ № 4 Кусто,http://www.ecolecousteau.ru/,30.238080,59.931625
4,1334077629,"Россия, Санкт-Петербург, набережная канала Гри...",ГБОУ СОШ № 256,http://sc.adm-edu.spb.ru/256,30.309037,59.926051


In [8]:
courses.head()

,address,id,name,url,long,lat
0,"Россия, Санкт-Петербург, проспект Испытателей, 39",117120638912,100 баллов,http://100баллов-спб.рф/,30.264269,60.007920
1,"Россия, Санкт-Петербург, Невский проспект, 80",1028074622,101 Балл,https://www.101ball.ru/,30.349208,59.932718
2,"Россия, Санкт-Петербург, улица Восстания, 11",224351539377,ABCDiplom,https://abcdiplom.com/,30.360727,59.933761
3,"Россия, Санкт-Петербург, Каменноостровский про...",182868913167,Advantage English Language Center,http://advantagepro.ru/,30.310741,59.966329
4,"Россия, Санкт-Петербург, улица Чайковского, 38/9",1244045040,American Kids Center ®,http://akcent.spb.ru/,30.358646,59.946689


In [10]:
g_courses = gpd.GeoDataFrame(courses.copy(), geometry=gpd.points_from_xy(courses.long, courses.lat))
g_schools = gpd.GeoDataFrame(schools.copy(), geometry=gpd.points_from_xy(schools.long, schools.lat))
g_courses.head()

,address,id,name,url,long,lat,geometry
0,"Россия, Санкт-Петербург, проспект Испытателей, 39",117120638912,100 баллов,http://100баллов-спб.рф/,30.264269,60.007920,POINT (30.26427 60.00792)
1,"Россия, Санкт-Петербург, Невский проспект, 80",1028074622,101 Балл,https://www.101ball.ru/,30.349208,59.932718,POINT (30.34921 59.93272)
2,"Россия, Санкт-Петербург, улица Восстания, 11",224351539377,ABCDiplom,https://abcdiplom.com/,30.360727,59.933761,POINT (30.36073 59.93376)
3,"Россия, Санкт-Петербург, Каменноостровский про...",182868913167,Advantage English Language Center,http://advantagepro.ru/,30.310741,59.966329,POINT (30.31074 59.96633)
4,"Россия, Санкт-Петербург, улица Чайковского, 38/9",1244045040,American Kids Center ®,http://akcent.spb.ru/,30.358646,59.946689,POINT (30.35865 59.94669)


In [11]:
schools_listings = gpd.sjoin(g_schools, data, op='within')
schools_listings

/home/eugene/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


,id,address,name,url,long,lat,geometry,index_right,NAME
0,1022453690,"Россия, Санкт-Петербург, 13-я линия Васильевск...",ГБОУ СОШ № 5,http://sch5.spb.ru/,30.272534,59.939229,POINT (30.27253 59.93923),51,округ № 7
24,1021195327,"Россия, Санкт-Петербург, Кадетская линия, 3",ГБОУ СОШ № 35 с углубленным изучением английск...,http://www.sch35.ru/,30.290248,59.942468,POINT (30.29025 59.94247),51,округ № 7
27,1004889394,"Россия, Санкт-Петербург, 5-я линия Васильевско...",Школа № 21 им. Э. П. Шаффе,http://spbschool21.ru/,30.284589,59.940630,POINT (30.28459 59.94063),51,округ № 7
95,1063831933,"Россия, Санкт-Петербург, 10-я линия Васильевск...",Средняя общеобразовательная школа № 27 имени И...,http://s27spb.ru/,30.281568,59.935556,POINT (30.28157 59.93556),51,округ № 7
104,1112607739,"Россия, Санкт-Петербург, 19-я линия Васильевск...",ГБОУ СОШ № 17 Василеостровского района Санкт-П...,http://www.school17vo.narod.ru/,30.264897,59.936966,POINT (30.26490 59.93697),51,округ № 7
...,...,...,...,...,...,...,...,...,...
505,206940493278,"Россия, Санкт-Петербург, 2-я Комсомольская ули...",ГБОУ Начальной школы - детского сада № 678 Кра...,http://school678.ucoz.net/,30.144795,59.834108,POINT (30.14479 59.83411),4,округ Константиновское
537,7297203945,"Россия, Санкт-Петербург, улица Пограничника Га...",Спортивная школа Красносельского района Санкт-...,None,30.147919,59.832475,POINT (30.14792 59.83248),4,округ Константиновское
764,1070056549,"Россия, Санкт-Петербург, улица Тамбасова, 26",Гимназия № 505,http://gymnase505.krsl.gov.spb.ru/,30.157217,59.831828,POINT (30.15722 59.83183),4,округ Константиновское
495,1046880072,"Россия, Санкт-Петербург, Колпинский район, пос...",Школа № 621,http://621.metallostroy.ru/,30.563378,59.813504,POINT (30.56338 59.81350),0,Металлострой


In [12]:
courses_listings = gpd.sjoin(g_courses, data, op='within')
courses_listings

/home/eugene/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


,address,id,name,url,long,lat,geometry,index_right,NAME
0,"Россия, Санкт-Петербург, проспект Испытателей, 39",117120638912,100 баллов,http://100баллов-спб.рф/,30.264269,60.007920,POINT (30.26427 60.00792),31,округ Озеро Долгое
14,"Россия, Санкт-Петербург, проспект Авиаконструк...",54275663755,BrainPlane,https://brain-plane.ru/,30.246976,60.011068,POINT (30.24698 60.01107),31,округ Озеро Долгое
68,"Санкт-Петербург, Комендантский просп., 13",1350099498,Speak Center,None,30.259295,60.009296,POINT (30.25930 60.00930),31,округ Озеро Долгое
82,"Санкт-Петербург, Комендантский проспект, 12к1",1418733110,UchuEnglish.com,http://uchuenglish.com/,30.257415,60.008095,POINT (30.25742 60.00809),31,округ Озеро Долгое
111,"Россия, Санкт-Петербург, проспект Королёва, 31к1",1258186402,Английский язык Все включено и все включены,None,30.263703,60.014342,POINT (30.26370 60.01434),31,округ Озеро Долгое
...,...,...,...,...,...,...,...,...,...
362,"Россия, Санкт-Петербург, улица Зины Портновой, 54",1529414407,Центр помощи студентам Заказ на 5,http://www.zakazna5.ru/,30.256725,59.848411,POINT (30.25672 59.84841),35,округ Княжево
400,"Россия, Санкт-Петербург, Ленинский проспект, 1...",1046582246,Юниум,https://unium.ru/,30.260043,59.852900,POINT (30.26004 59.85290),35,округ Княжево
317,"Россия, Санкт-Петербург, Пискарёвский проспект...",6445449561,Точная наука,http://точнаянаука.рф/,30.413542,59.969668,POINT (30.41354 59.96967),28,округ Полюстрово
343,"Россия, Санкт-Петербург, Балканская площадь, 5И",11513518931,Формула Знаний,https://formulaznaniy.ru/,30.380129,59.829142,POINT (30.38013 59.82914),14,Балканский округ


In [13]:
grouped_sch = schools_listings.groupby('NAME').size()

schools_df = grouped_sch.to_frame().reset_index()
schools_df.columns = ['NAME', 'listings_count']
schools_df.rename(columns={"listings_count": "schools_count"}, inplace = True)

schools_df

,NAME,schools_count
0,Адмиралтейский округ,4
1,Балканский округ,9
2,Васильевский округ,9
3,Введенский округ,5
4,Владимирский округ,17
...,...,...
77,округ № 65,24
78,округ № 7,15
79,округ № 72,13
80,округ № 75,8


In [14]:
grouped_cou = courses_listings.groupby('NAME').size()

courses_df = grouped_cou.to_frame().reset_index()
courses_df.columns = ['NAME', 'listings_count']
courses_df.rename(columns={"listings_count": "courses_count"}, inplace = True)

courses_df

,NAME,courses_count
0,Адмиралтейский округ,3
1,Балканский округ,1
2,Васильевский округ,3
3,Введенский округ,3
4,Владимирский округ,14
...,...,...
69,округ № 65,12
70,округ № 7,4
71,округ № 72,1
72,округ № 75,2


In [15]:
merged_data = data.merge(schools_df, on='NAME', how='outer')
merged_data = merged_data.merge(courses_df, on='NAME', how='outer')
merged_data.fillna({'schools_count':0, 'courses_count':0}, inplace = True)

merged_data['coef'] = merged_data['courses_count']/merged_data['schools_count']

merged_data.head(10)

,NAME,geometry,schools_count,courses_count,coef
0,Металлострой,"POLYGON ((30.57063 59.81767, 30.57122 59.81742...",2.0,0.0,0.000000
1,округ Юго-Запад,"POLYGON ((30.23920 59.86393, 30.22867 59.85674...",10.0,2.0,0.200000
2,округ Московская застава,"POLYGON ((30.33814 59.90507, 30.33813 59.90500...",14.0,9.0,0.642857
3,округ Урицк,"POLYGON ((30.19716 59.84589, 30.20374 59.84576...",5.0,1.0,0.200000
4,округ Константиновское,"POLYGON ((30.12778 59.80916, 30.12769 59.80907...",7.0,0.0,0.000000
5,округ Сосновая поляна,"POLYGON ((30.16130 59.83420, 30.16122 59.83382...",7.0,2.0,0.285714
6,Петро-Славянка,"POLYGON ((30.54713 59.81369, 30.52804 59.79749...",0.0,0.0,NaN
7,округ Чёрная речка,"POLYGON ((30.25651 59.98305, 30.25650 59.98317...",13.0,8.0,0.615385
8,округ Комендантский аэродром,"POLYGON ((30.31596 60.01239, 30.31638 60.01164...",9.0,7.0,0.777778
9,округ № 65,"POLYGON ((30.19328 59.98158, 30.19332 59.98143...",24.0,12.0,0.500000


In [16]:
merged_data['coef'] = merged_data['coef'].fillna(5)
merged_data.head(10)

,NAME,geometry,schools_count,courses_count,coef
0,Металлострой,"POLYGON ((30.57063 59.81767, 30.57122 59.81742...",2.0,0.0,0.000000
1,округ Юго-Запад,"POLYGON ((30.23920 59.86393, 30.22867 59.85674...",10.0,2.0,0.200000
2,округ Московская застава,"POLYGON ((30.33814 59.90507, 30.33813 59.90500...",14.0,9.0,0.642857
3,округ Урицк,"POLYGON ((30.19716 59.84589, 30.20374 59.84576...",5.0,1.0,0.200000
4,округ Константиновское,"POLYGON ((30.12778 59.80916, 30.12769 59.80907...",7.0,0.0,0.000000
5,округ Сосновая поляна,"POLYGON ((30.16130 59.83420, 30.16122 59.83382...",7.0,2.0,0.285714
6,Петро-Славянка,"POLYGON ((30.54713 59.81369, 30.52804 59.79749...",0.0,0.0,5.000000
7,округ Чёрная речка,"POLYGON ((30.25651 59.98305, 30.25650 59.98317...",13.0,8.0,0.615385
8,округ Комендантский аэродром,"POLYGON ((30.31596 60.01239, 30.31638 60.01164...",9.0,7.0,0.777778
9,округ № 65,"POLYGON ((30.19328 59.98158, 30.19332 59.98143...",24.0,12.0,0.500000


In [17]:
merged_data['coef'].describe()

count    83.000000
mean      0.548888
std       0.723911
min       0.000000
25%       0.166667
50%       0.333333
75%       0.671429
max       5.000000
Name: coef, dtype: float64

In [18]:
# Convert the dataframe with data from the previous step into json to forward it to Bokeh:
merged_json = json.loads(merged_data.to_json())
json_merged_data = json.dumps(merged_json)

In [19]:
# Creating an interactive map with this data with Bokeh:
geosource = GeoJSONDataSource(geojson = json_merged_data)

# Creating a map and setting the values that will be displayed when put the cursor over a certain area
# Explicity specifying a decimal format for displaying to avoid displaying some coefficient values in a scientific format (like '5.263e-2'): @coef{(0.0000)}
p = figure(title = 'Тепловая карта районов', plot_height = 700 , plot_width = 700, toolbar_location = None, tooltips=[("Name", "@NAME"),("Coefficient", "@coef{(0.0000)}")])# убираем отображение линий сетки
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# set the color scheme of the heat map and the number of color gradation
palette = brewer['RdBu'][5]
# set the poles of the color scale and its display in the legend
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1,  nan_color = '#d9d9d9')
color_bar = ColorBar(color_mapper=color_mapper,label_standoff=8,width = 500,height = 20, border_line_color='black',location = (0,0),orientation ='horizontal')
p.add_layout(color_bar, 'below')

# indicating that the districts will be colored according to the value of the 'coef' column.
p.patches('xs','ys', source = geosource,fill_color = {'field' : 'coef', 'transform' : color_mapper},line_color = 'black', line_width = 0.25, fill_alpha = 1)

output_notebook()
show(p)

Loading BokehJS ...

In [20]:
coursedots = p.circle('long', 'lat', source = courses, color = 'yellow', size = 4, alpha = 0.9)
schooldots = p.circle('long', 'lat', source = schools, color = 'green', size = 4, alpha = 0.9)
p.add_tools(HoverTool(renderers = [coursedots,schooldots],tooltips = [('Название', '@name'), ('Адрес', '@address')]))

layout = column(p)
show(layout)

In [21]:
attractive = merged_data[merged_data['coef']< 0.2].sort_values(by=['coef'])
attractive

,NAME,geometry,schools_count,courses_count,coef
0,Металлострой,"POLYGON ((30.57063 59.81767, 30.57122 59.81742...",2.0,0.0,0.000000
75,округ Рыбацкое,"POLYGON ((30.48068 59.85472, 30.48053 59.85520...",7.0,0.0,0.000000
65,Екатерингофский округ,"POLYGON ((30.29513 59.90646, 30.29510 59.90643...",10.0,0.0,0.000000
58,округ Красненькая речка,"POLYGON ((30.25357 59.85882, 30.25056 59.85736...",5.0,0.0,0.000000
32,округ Морские ворота,"POLYGON ((30.25023 59.90129, 30.24823 59.90014...",1.0,0.0,0.000000
79,Ивановский округ,"POLYGON ((30.43989 59.89175, 30.44436 59.89244...",3.0,0.0,0.000000
4,округ Константиновское,"POLYGON ((30.12778 59.80916, 30.12769 59.80907...",7.0,0.0,0.000000
11,округ Лахта-Ольгино,"POLYGON ((30.17854 59.99030, 30.17862 59.98998...",4.0,0.0,0.000000
24,округ Пороховые,"POLYGON ((30.44636 59.96046, 30.45218 59.96063...",19.0,1.0,0.052632
54,Финляндский округ,"POLYGON ((30.40693 59.95894, 30.40321 59.95703...",15.0,1.0,0.066667


In [22]:
attractive_json = json.loads(attractive.to_json())
json_attractive = json.dumps(attractive_json)

In [23]:
geosource2 = GeoJSONDataSource(geojson = json_attractive)


d = figure(title = 'Привлекательные районы', plot_height = 700 , plot_width = 700, toolbar_location = None, 
          tooltips=[
         ("Name", "@NAME"),
         ("Coefficient", "@coef{(0.0000)}")])

# Makes it so there are no gird lines
d.xgrid.grid_line_color = None
d.ygrid.grid_line_color = None

palette = brewer['RdBu'][5]
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1,  nan_color = '#d9d9d9')
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20, border_line_color='black',location = (0,0), orientation ='horizontal')
d.add_layout(color_bar, 'below')

d.patches('xs','ys', source = geosource2,fill_color = {'field'     :'coef', 'transform' : color_mapper},
         line_color = 'black', line_width = 0.25, fill_alpha = 1)

output_notebook()
show(d)

Loading BokehJS ...

In [24]:
not_attractive = merged_data[merged_data['coef']> 0.6].sort_values(by=['coef'], ascending = False)
not_attractive

,NAME,geometry,schools_count,courses_count,coef
6,Петро-Славянка,"POLYGON ((30.54713 59.81369, 30.52804 59.79749...",0.0,0.0,5.000000
37,округ Лиговка-Ямская,"POLYGON ((30.36101 59.93104, 30.36154 59.93083...",4.0,12.0,3.000000
12,Дворцовый округ,"POLYGON ((30.33489 59.95015, 30.33741 59.94632...",7.0,17.0,2.428571
66,Сенной округ,"POLYGON ((30.30818 59.92645, 30.30569 59.92679...",8.0,16.0,2.000000
29,округ Аптекарский остров,"POLYGON ((30.31175 59.96558, 30.31164 59.96551...",10.0,15.0,1.500000
73,Парголово,"POLYGON ((30.27433 60.06872, 30.27426 60.06869...",10.0,14.0,1.400000
31,округ Озеро Долгое,"POLYGON ((30.27848 59.99354, 30.27842 59.99317...",14.0,18.0,1.285714
59,округ Измайловское,"POLYGON ((30.29818 59.90827, 30.29820 59.90864...",10.0,12.0,1.200000
15,округ № 78,"POLYGON ((30.31239 59.93739, 30.31235 59.93727...",9.0,10.0,1.111111
30,округ Новоизмайловское,"POLYGON ((30.31072 59.84133, 30.31117 59.83340...",11.0,12.0,1.090909
